In [1]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
from IPython.display import display, Markdown


ModuleNotFoundError: No module named 'qiskit_aer'

In [7]:
simulator = AerSimulator()

In [8]:
def create_bb84_circuit(bits, bases):
    num_qubits = len(bits)
    qc = QuantumCircuit(num_qubits, num_qubits)

    for i in range(num_qubits):
        if bases[i] == 1:
            qc.h(i)
        if bits[i] == 1:
            qc.x(i)
        if bases[i] == 1:
            qc.h(i)
        qc.barrier()

        return qc, bits, bases

In [9]:
def visualize_qubit_creation(qc):
    display(Markdown("Circuit Diagram: Qubit State After Creation"))
    state = Statevector.from_instruction(qc)
    plot_bloch_multivector(state, title = "Qubit States After Creation")
    display(qc.draw(output = 'mpl'))